In [43]:
# import os
# os.environ["GOOGLE_API_KEY"] = ""

In [2]:
!pip install -q langchain-google-genai langchain-core requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
# tool binding

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [16]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68a1-32ec-7953-a450-99ed4f6f8bf7-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [17]:
llm_with_tools = llm.bind_tools([multiply])

In [18]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, how about you?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68a1-c23a-76f1-8a5d-1eff3b7ff146-0', usage_metadata={'input_tokens': 58, 'output_tokens': 10, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}})

In [19]:
query = HumanMessage('can you multiply 3 with 1000')

In [20]:
messages = [query]

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [25]:
result = llm_with_tools.invoke(messages) #this will allow the llm to extract all the details from the message and store it in a dictionary

In [39]:
result=llm_with_tools.invoke("multiply 3 by 1000")
print(result)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3, "b": 1000}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b68af-f3bf-7fb2-a8cb-632d4aaec08f-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '837df542-543d-4449-a831-388625151010', 'type': 'tool_call'}] usage_metadata={'input_tokens': 63, 'output_tokens': 21, 'total_tokens': 84, 'input_token_details': {'cache_read': 0}}


In [32]:
print(type(result))

<class 'langchain_core.messages.ai.AIMessage'>


In [26]:
messages.append(result)

In [27]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 1000, "a": 3}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68a2-c0e7-7a82-a086-73190cb826b0-0', tool_calls=[{'name': 'multiply', 'args': {'b': 1000, 'a': 3}, 'id': '426293b7-c137-440e-8050-518110406620', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 86, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68a5-59bb-7120-8d1d-44f78a16e438-0', usage_metadata={'input_tokens': 87, 'output_tokens': 0, 'total_tokens': 87, 'input_token_details': {

In [34]:
tool_result = multiply.invoke(messages[1].tool_calls[0])

In [35]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='426293b7-c137-440e-8050-518110406620')

In [40]:
messages.append(tool_result)

In [ ]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RxxH1pPDylDECUwpRe7MXkJi', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 63, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BR8rS3DNc8cckcVLJMmBDxHENKUlV', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8035ac83-7820-4681-b8c0-1d15aa24ca77-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_RxxH1pPDylDECUwpRe7MXkJi', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19

In [41]:
llm_with_tools.invoke(messages)

AIMessage(content='3000', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b68b7-bf33-7070-894a-4b69638b7d24-0', usage_metadata={'input_tokens': 110, 'output_tokens': 4, 'total_tokens': 114, 'input_token_details': {'cache_read': 0}})

In [42]:
print(llm_with_tools.invoke(messages).content)

3000


In [38]:
messages.append(
    HumanMessage(content="What is the final answer?")
)

print(llm_with_tools.invoke(messages).content)

The final answer is 3000.


In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [ ]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4741}

In [ ]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [ ]:
# tool binding
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_PKL8v7zwmphzNel0MnvjjGvY',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_vRdld30yHTKFGcTQPumgzH5u',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PKL8v7zwmphzNel0MnvjjGvY', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_vRdld30yHTKFGcTQPumgzH5u', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 123, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BR8rovupYuNnIbGO41TOO3knQ2Cr9', 'finish_reason': 'too

In [ ]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.0117. \n\nBased on this conversion factor, 10 INR is equal to 0.117 USD.'

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to help! How can I assist you today?

> Finished chain.
